In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

## 0. Getting Started

In [2]:
feature_reduction = 'PCA16'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', '2', feature_reduction=feature_reduction, classes=classes)
X1_test, X0_test = [], []
for i in range(len(X_test)):
    if Y_test[i] == 1:
        X1_test.append(X_test[i])
    else:
        X0_test.append(X_test[i])
X1_test, X0_test = torch.tensor(X1_test), torch.tensor(X0_test)

2022-12-18 20:53:22.688658: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-18 20:53:22.690102: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 10. Tune using inter_op_parallelism_threads for best performance.
/opt/anaconda3/envs/QC/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646835196/work/torch/csrc/utils/tensor_new.cpp:233.)
  # Remove the CWD from sys.path while we load stuff.


Calculate the Trace Distance Before training the embedding

In [15]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev, interface="torch")
def distance_circuit1(inputs): 
    qml.AmplitudeEmbedding(inputs, wires=range(4), normalize=True)
    return qml.density_matrix(wires=range(4))

class Distance(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1_distance = qml.qnn.TorchLayer(distance_circuit1, weight_shapes={})
    
    def forward(self, x1, x0, measure):
        rhos1 = self.qlayer1_distance(x1)
        rhos0 = self.qlayer1_distance(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0

        if measure == "Trace":
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))
        elif measure == "Hilbert-Schmidt":
            return 0.5 * torch.real(torch.trace(rho_diff @ rho_diff))
        

D = Distance()
D_trace = D(X1_test, X0_test, "Trace")
D_HS = D(X1_test, X0_test, "Hilbert-Schmidt")
print(f"Trace Distance before: {D_trace}\n")
print(f"Hilbert Schmidt distance before: {D_HS}")

/opt/anaconda3/envs/QC/lib/python3.7/site-packages/pennylane/qnn/torch.py:328: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1666646835196/work/aten/src/ATen/native/Copy.cpp:250.)
  return self.qnode(**kwargs).type(x.dtype)


Trace Distance before: 0.542850911617279

Hilbert Schmidt distance before: 0.22676578164100647


## 1. Distances After the Training

In [16]:
class Distance_After(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qlayer1_distance = qml.qnn.TorchLayer(distance_circuit1, weight_shapes={})
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16,32),
            nn.ReLU(),
            nn.Linear(32,32),
            nn.ReLU(),
            nn.Linear(32,16)
        )
    
    def forward(self, x1, x0, measure):
        x1 = self.linear_relu_stack(x1)
        x0 = self.linear_relu_stack(x0)
        rhos1 = self.qlayer1_distance(x1)
        rhos0 = self.qlayer1_distance(x0)
        rho1 = torch.sum(rhos1, dim=0) / len(x1)
        rho0 = torch.sum(rhos0, dim=0) / len(x0)
        rho_diff = rho1 - rho0

        if measure == "Trace":
            eigvals = torch.linalg.eigvals(rho_diff)
            return 0.5 * torch.real(torch.sum(torch.abs(eigvals)))
        elif measure == "Hilbert-Schmidt":
            return 0.5 * torch.real(torch.trace(rho_diff @ rho_diff))

In [18]:
Model_Amplitude_PATH = []
for i in range(5):
    Model_Amplitude_PATH.append(f"/Users/tak/Github/QEmbedding/Results/earlystop 10 experiments/experiment{i+1}/Model Amplitude/Model_Amplitude.pt")

Model_Amplitude_Trace_Distances, Model_Amplitude_HS_Distances = np.array([]), np.array([])
for path in Model_Amplitude_PATH:
    Model = Distance_After()
    Model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    with torch.no_grad():
        trace_distance = Model(X1_test, X0_test, 'Trace')
        HS_distance = Model(X1_test, X0_test, 'Hilbert-Schmidt')
    Model_Amplitude_Trace_Distances = np.append(Model_Amplitude_Trace_Distances, trace_distance)
    Model_Amplitude_HS_Distances = np.append(Model_Amplitude_HS_Distances, HS_distance)
print(f"Trace Distance After training with Model_Amplitude: {Model_Amplitude_Trace_Distances.mean()} ± {Model_Amplitude_Trace_Distances.std()}")
print(f"Hilbert Schmidt distance After training with Model_Amplitude: {Model_Amplitude_HS_Distances.mean()} ± {Model_Amplitude_HS_Distances.std()}")

Trace Distance After training with Model_Amplitude: 0.9526245474815369 ± 0.001617120687990123
Hilbert Schmidt distance After training with Model_Amplitude: 0.8807974100112915 ± 0.006995544806932898
